# LangChain Use Cases

This document dives straight into implementing [LangChain use case](https://docs.langchain.com/docs/category/use-cases) using a step-by-step approach, and hopefully inspire you to build.

## Summarization

Langchain summarization is one of the most common usage of LangChain and LLMs. It can summarize any amount of text or documentations, including the ones that exceeds the token limit (currently set at `4096 tokens` for `gpt-35-turbo`), which roughly equates to `3000 words`. 

The technique used is similar to the concept of `sliding window`, which a fixed size window, usually under the max context window limit, is used to chunk the the long document into smaller pieces, then summarize the content recursively to produce the final summary using mapreduce.

You can use it to summarize not only text, books, documents, audios, social media threads, and etc. Some use cases including: _articles_, _research pagers_, _legal and financial documents_, _transcripts_, _chat history_, _customer interactions_, _product reviews_, _podcasts_, _twitter threads_ and more.

In [4]:
import dotenv
import os
import pdfplumber
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(temperature = 0, openai_api_key = openai_api_key)

Now, let's load up `The Tragedy of Hamlet`

In [5]:
with pdfplumber.open("hamlet.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
        
print(text[:300])

The Tragedy of Hamlet, Prince of
Denmark
ASCIItextplacedinthepublicdomainbyMobyLexicalTools,1992.SGMLmarkupbyJonBosak,
1992-1994.XMLversionbyJonBosak,1996-1999.SimplifiedXMLversionbyMaxFroumentin,2001.The
XMLmarkupinthisversionisCopyright©1999JonBosak.Thisworkmayfreelybedistributedoncondition
thatit


By outputting the total number of tokens of the book with [get_num_tokens](https://python.langchain.com/en/latest/reference/modules/llms.html#langchain.llms.OpenAI.get_num_tokens), you can clearly see it exceeds the context window limit.

In [7]:
n_tokens = llm.get_num_tokens(text)

print(f"total number of tokens: {n_tokens}")

total number of tokens: 53929


The process starts by `splitting` the text into multiple parts by `chunk size` using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html), with a `coverlap` of each adjacent chunk of text.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=2000, chunk_overlap=200, length_function = len)
docs = text_splitter.create_documents([text])

print(f"number of docs: {len(docs)}")

number of docs: 100


Now the docs are ready, we can then load up a `chain` to do produce the sum of sums.

In [12]:
chain = load_summarize_chain(llm = llm, chain_type = 'map_reduce')
result = chain.run(docs)

print(result)

 In The Tragedy of Hamlet, Prince of Denmark, Hamlet is on a quest for revenge against his uncle, Claudius, who has usurped the throne. After encountering a ghostly figure, Hamlet learns that his father was murdered by his uncle and is determined to avenge his death. King Claudius and Lord Polonius devise a plan to spy on Hamlet and Ophelia, and Hamlet kills Polonius. Laertes returns from France and challenges Hamlet to a fencing match, during which Laertes wounds Hamlet with a poisoned sword. Hamlet stabs King Claudius with a poisoned sword and forces him to drink a poisoned potion. Prince Fortinbras arrives and orders that the bodies be placed on a stage for all to see, and then claims his rights of memory in the kingdom.


When instanciating the `chain`, you can also add `Verbose = True` to see the steps LangChain took before producing the final summary, i.e. generating a summary per each chunk of text, and produce the final summary by aggregating the 100 summaries with map reduce.

This is a powerful technique to overcome the token limitation. Even though more powerful models will be released in the future to support more tokens, such as Claude now supports [100K tokens](https://www.anthropic.com/index/100k-context-windows) as of 11 May 2023, biggest of the its kind. There is not guaranteed better performance, accuracy or cost-effectiveness.

Moreover, you can [parallelise the calls](https://github.com/hwchase17/langchain/issues/1073) to super charge summarization using `batch_size` when instanciating LLMs.

## Question Answering Over Documents

Similiar to how humans answer questions, first you need to provide some context to LLMs. The context can be in many different formats, such as text documents, SQL database, APIs, etc. For the purpose of this exercise, we will focus on dealing with text documents input as context. 

However, as the context grows, it naturally exceeds the token limit (again, just like in the case of summarization), and it gets harder and harder to give accurate answer quickly. The problem is solved by converting the context and your question into [embeddings](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings), basically a list of vector value representaiton of information; then find out a few results that are closely related to your question before giving out a final answer. The process to do the comparison is via an algorithm called [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). If you are getting confused by the terminologies, don't worry. The formular is as following: 

`llm(context + question) = your answer`.

With this capability, you can _chat with your documents_, _ask questions to papers_, _create study guides_, _reference medical information_ and more.

In [15]:
from langchain import OpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings

llm = OpenAI(temperature = 0, openai_api_key = openai_api_key)

Now, let's load `The Tragedy of Hamlet` again.

In [47]:
loader = PyMuPDFLoader("hamlet.pdf")
doc = loader.load()

print(f"number of pages in the doc: {len(doc)}")
total_chars = sum([len(page.page_content) for page in doc])
print(f"number of characters in the doc: {total_chars}")

number of pages in the doc: 142
number of characters in the doc: 179843


Now, we can split the PDF into chunks based on our definition.

In [48]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)
docs = text_splitter.split_documents(doc)

print(f"number of splitted pages: {len(docs)}")
print(f"average number of characters in each page: {total_chars / len(docs):,.0f}")

number of splitted pages: 143
average number of characters in each page: 1,258


Then convert the documents into embeddings by calling OpenAI API Embedding engine, and store the result in a local vector store called `FAISS`. You can choose from a number of supported [vector stores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) (local or remote).

In [22]:
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)
knowledge_base = FAISS.from_documents(docs, embeddings)

Create a retrieval QA engine to perform queries.

In [53]:
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "refine", retriever = knowledge_base.as_retriever())
query = """What is the subject of Hamlet's second soliloquy, the famous "To be or not to be" speech?"""
qa.run(query)

'\n\nThe subject of Hamlet\'s second soliloquy, the famous "To be or not to be" speech, is the contemplation of life and death and the decision of whether to take action against the troubles of life or to accept them. Hamlet is reflecting on the consequences of his inaction in the face of his father\'s death and his mother\'s remarriage, and is considering the idea of suicide as a way to escape his suffering. He is also questioning the value of life and death, and whether it is better to endure the pain of life or to end it. He is further considering the implications of his actions, and whether his life is worth the risk of taking action against his troubles. He is also questioning the power of death, and whether it is better to accept death or to fight against it. Additionally, Hamlet is questioning the nature of ambition and the idea of being "bounded in a nut shell" and whether it is possible to be content with one\'s life despite the troubles and suffering that come with it.'

By now, you should have a working example of QA over documents. You should also notice there are a number of similarities comparing to `summarization`, such as the way the context was loaded, splitted, before passing onto the `chain`.

You may also have noticed different `chain types` specification used, and there is a good reason for that. Before explaining further, understand that LangChain chain types are basically different ways you can connect LLMs. Here are the supported chain types:
- `map_reduce`: It separates texts into batches, feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.
- `map_rerank`: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.
- `refine`: It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.
- `stuff`: The default chain type that uses ALL of the text from the documents in the prompt.

Because the chain type will actually affect the result, I highly recommend you try different types, and figure out which works best for your scenario.

## Extraction

LLMs are very good at extracting structured information out of unstructured text, whehter that's extracting one or multiple stuctured row from text to insert into database from a sentence, or extracting the correct API params from a user query.

In essence, it is about working with [OutputParsers](https://python.langchain.com/en/latest/modules/prompts/output_parsers.html), which is responsible for specifying the schema a LLM should respond in, and then parsing their raw-text output into that structured format.

Without further ado, let's dive straight into an example.

In [56]:
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

chat_model = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo", openai_api_key = openai_api_key)

This step is to define the schema, or more specicially, format instructions, to be added to the prompt, so that LLMs can act accordingly.

In [62]:
response_schemas = [
    ResponseSchema(name="author", description="The name of the author"),
    ResponseSchema(name="books", description="The books published")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema:

```json
{
	"author": string  // The name of the author
	"books": string  // The books published
}
```


In [77]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            """Extract the author and books.\n\n{format_instructions}\n{user_prompt}""")  
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)

In [78]:
query = prompt.format_prompt(user_prompt="I recently read the book The Tragedy of Hamlet written by William Shakespeare")
print (query.messages[0].content)

Extract the author and books.

The output should be a markdown code snippet formatted in the following schema:

```json
{
	"author": string  // The name of the author
	"books": string  // The books published
}
```
I recently read the book The Tragedy of Hamlet written by William Shakespeare


In [79]:
output = chat_model(query.to_messages())
result = output_parser.parse(output.content)

print (result)
print (type(result))

{'author': 'William Shakespeare', 'books': 'The Tragedy of Hamlet'}
<class 'dict'>


This is pretty cool example of data extraction and sanitisation, however, it is also quite basic. In real life scenario, you are likely to be dealing with more complex data and schema, such as nested structures. In which case, make sure to check out [Kor](https://eyurtsev.github.io/kor/), which provides the next level functionalities. 

## Querying Tabular Data

Aside from unstructured data, there is also a lot of structured data sitting in CSVs, SQL, whether that's financial data, report or models or gazillions of day-to-day data that every company has. LLMs can deal with that as well. 

It requires a bit of prep in order to demonstrate this:
- To start with, I'll be using a `SQLite` database which basically stores `Goodreads Top 100 Classics`, this data comes from [kaggle](https://www.kaggle.com/datasets/notkrishna/goodreads-top-100-classical-books-of-all-time). Make sure you have SQLite installed or run `brew install sqlite3`.
- Use [this](https://www.convertcsv.com/csv-to-sql.htm) to convert CSV data into a SQL INSERT script, results are kept in `goodreads.sql`.
- Run the command to insert the data into a database called `goodreads.db`.
```
sqlite3 goodreads.db
sqlite> .read goodreads.db
```

In [96]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

llm = OpenAI(temperature = 0, openai_api_key = openai_api_key)

This step will load up the data for the chain.

In [97]:
sqlite_db_path = "goodreads.db"
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

Then we can query the database using natural language, and I'll enable `verbose` mode so that you can see the chain of thoughts there.

In [99]:
db_chain = SQLDatabaseChain(llm = llm, database = db, verbose = True)
db_chain.run("How many books achieved an average rating above 4?")



> Entering new SQLDatabaseChain chain...
How many books achieved an average rating above 4? 
SQLQuery: SELECT COUNT(*) FROM goodreads WHERE avg_rating > 4;
SQLResult: [(100,)]
Answer: 100 books achieved an average rating above 4.
> Finished chain.


' 100 books achieved an average rating above 4.'

Amazing! By using `verbose` mode, you can actually obverse the SQL query being constructed to extract the data based on our question, and the output in natural language as well.

Let's run the query using `pandas` to verify the result, in case any hallucination in the LLM.

In [101]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(sqlite_db_path)
query = "SELECT COUNT(*) FROM goodreads WHERE avg_rating > 4;"
df = pd.read_sql_query(query, conn)
conn.close()

print(df.iloc[0,0])

100


And the result checks out! But that's not it. To query larger databases and more complex schemas, we will need to introduce the use of `agents`, which typically involves running multiple sequential queries or error recovery. You can checkout the [agent_toolkits](https://github.com/hwchase17/langchain/tree/master/langchain/agents/agent_toolkits) for more details or I'll cover this concept when introducing `Agents`.

## Code Understanding

This is probably one of the most sought after features every developer or anyone who's interested in building has been dreaming for. LangChain can parse GitHub repos and generate new code using the mechnisms demonstrated so far, including `embeddings`, `vector stores`, `conversational retriever chain` and `LLMs`.

Without paying for [GitHub Copilot](https://github.com/features/copilot), why not see what you can do with LLMs first?

In [103]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI 

llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)

Here we do the embeddings again in preparation for loading the project as context.

In [107]:
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)

Scan through the whole codebase, the example here is a library called [pandas-ai](https://github.com/gventuri/pandas-ai), which is a clever wrapper around LLMs and the well-known [Pandas](https://github.com/pandas-dev/pandas).

In [109]:
project = "pandas-ai"
docs = []

for path, dirs, files in os.walk(project):
    for file in files:
        try: 
            loader = TextLoader(os.path.join(path, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass
        
print(f"number of docs loaded: {len(docs)}")

number of docs loaded: 133


Now we create the embeddings and store them locally.

In [111]:
knowledge_base = FAISS.from_documents(docs, embeddings)
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = knowledge_base.as_retriever())

Now the fun begins, we gonna test whether LLM really understands the code and can help us code.

In [112]:
query = "How can I use Google PaLM2 as the LLM for queries?"
output = qa.run(query)

print(output)

To use Google PaLM2 as the LLM for queries, you can use the `GooglePalm` class provided in the code. Here's an example of how you can use it:

```
from pandasai.llm.google_palm import GooglePalm

# Replace YOUR_API_KEY with your Google Cloud API key
llm = GooglePalm(api_key='YOUR_API_KEY', model='models/text_bison_001')
response = llm.call(instruction='Hello', value='world')

print(response)
```

You need to replace `YOUR_API_KEY` with your own Google Cloud API key. You can also change the value of the `model` parameter to use a different model if necessary.


In [113]:
query = "Please write the code to instanciate Google PaLM2 as LLM, and only respond with code."
output = qa.run(query)

print(output)

```
from pandasai.llm.google_palm import GooglePalm

llm = GooglePalm(api_key="your_api_key")
```


As you can see from the output, the result is pretty staggering. LLM plus LangChain can truly understand code, explain the functionalities, and it's able to produce some sensible code as a result of that. 

However, this is a very small sample in terms of its coding ability, you can use it to bootstrap work, but don't rely on it to code for you! Until we reach AGI, we still need a sound logical-minded person to give directions.

## Evaluation

By now, you probably have noticed that the output of LLMs can vary dramatically. Even with the same LLM, you can still expect different results for the same question every time you rerun. Lowering the temperature setting is one way to have more controlled and predictable output, however, that's not quite the point here.

Evaluation is the mechnism to quality control the pipeline, i.e. the output of applications, and make sure it does not suffer from any regression.

In [87]:
from langchain import OpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.evaluation.qa import QAEvalChain

llm = OpenAI(temperature = 0, openai_api_key = openai_api_key)

We need to prepare the data just like what we did in `Question Answering Over Documents`.

In [88]:
loader = PyMuPDFLoader("hamlet.pdf")
doc = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)
docs = text_splitter.split_documents(doc)
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)
knowledge_base = FAISS.from_documents(docs, embeddings)

Then prepare a list of questions and answers as test cases, which I picked a few from [this](hamlet_qna.pdf).

In [90]:
question_answers = [
    {"question" : "Where does the play take place?", "answer" : "Denmark"},
    {"question" : "What is the name of the castle?", "answer" : "Elsinore"},
    {"question" : "What are the first words spoken in the play?", "answer" : "Who's there?"},
    {"question" : "How has Ophelia died?", "answer" : "She has supposedly drowned (ambiguity surrounds her death)."},
    {"question" : "When was Hamlet written?", "answer" : "1600-1601"},
]

Create a `RetrievalQA chain`, the `input_key` tells the chain to look for the `question` key in the dict above, as the input prompt / query to the LLM.

In [91]:
chain = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = knowledge_base.as_retriever(), 
    input_key = "question"
)

This step is like executing the unit tests, it not only outputs the test asseration, but also the results for comparison.

In [94]:
predications = chain.apply(question_answers)
predications

[{'question': 'Where does the play take place?',
  'answer': 'Denmark',
  'result': ' The play takes place in Denmark.'},
 {'question': 'What is the name of the castle?',
  'answer': 'Elsinore',
  'result': ' The castle is Elsinore.'},
 {'question': 'What are the first words spoken in the play?',
  'answer': "Who's there?",
  'result': ' The first words spoken in the play are "Who\'s there?" by Francisco.'},
 {'question': 'How has Ophelia died?',
  'answer': 'She has supposedly drowned (ambiguity surrounds her death).',
  'result': ' Ophelia has drowned.'},
 {'question': 'When was Hamlet written?',
  'answer': '1600-1601',
  'result': ' Hamlet was written by William Shakespeare in the early 1600s.'}]

Once the execution result is produced, we call on `QAEvalChain` to evaluate the result.

In [95]:
eval_chain = QAEvalChain.from_llm(llm)
evaluation = eval_chain.evaluate(
    question_answers,
    predications,
    question_key = "question",
    answer_key = "answer",
    prediction_key = "result"
)
evaluation

[{'text': ' CORRECT'},
 {'text': ' CORRECT'},
 {'text': ' CORRECT'},
 {'text': ' CORRECT'},
 {'text': ' CORRECT'}]

Now the results are here, all answered correctly. Despite the exceptional outcome, I was expecting it to make some tiny mistakes, which would probably seem more human. I almost tempted to create some convoluted mind-twisting asserations, I am limited by my own creativity.

Another thing you might find interesting is: having QAEvalChain marking it own homework. And I am not entirely sure how that mechnism works underneath the hood. So either you should dig into it if you are interested or take it with a pinch of salt (at least for now, until I can find a self-consistent answer).